In [51]:
import os
import pandas as pd
from duckdb import query as queryFun
def sqldf(query):
    return queryFun(query).to_df()

In [180]:
def extraiAno(linha):
    texto = linha["Data"]
    return int(texto[0:4])

def extraiMes(linha):
    texto = linha["Data"]
    return int(texto[5:7])




def geraDFSummary(stringArquivo):

    with open(stringArquivo, "r", encoding="latin1") as file:
        linhas = file.readlines()[0:20]

    for linha in linhas:

        if "CODIGO" in linha:
            codigo = linha.replace("CODIGO (WMO):;", "").replace("\n", "")
        
        if "LATITUDE" in linha:
            latitude = float(linha.replace("LATITUDE:;", "").replace("\n", "").replace(",", "."))
        
        if "LONGITUDE" in linha:
            longitude = float(linha.replace("LONGITUDE:;", "").replace("\n", "").replace(",", "."))

        if "REGIAO" in linha:
            regiao = linha.replace("REGIAO:;", "").replace("\n", "")

        if "REGIÃO" in linha:
            regiao = linha.replace("REGIÃO:;", "").replace("\n", "")

        if "REGI?O" in linha:
            regiao = linha.replace("REGI?O:;", "").replace("\n", "")
        
        if "UF" in linha:
            uf = linha.replace("UF:;", "").replace("\n", "")

    df = pd.read_csv(stringArquivo, encoding="latin1", header=8, sep=";")

    df.columns

    stringData = "Data" if "Data" in df.columns else (
        "DATA (YYYY-MM-DD)"# if "DATA (YYYY-MM-DD)" in df.columns else 
    )

    df = df[[stringData, "UMIDADE RELATIVA DO AR, HORARIA (%)", "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)"]]


    df = df.rename(columns= {"UMIDADE RELATIVA DO AR, HORARIA (%)":"Umidade",
                        "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)":"Temperatura",
                        stringData:"Data"})
    df["Ano"] = df.apply(extraiAno, axis=1)
    df["Mes"] = df.apply(extraiMes, axis=1)

    df["Temperatura"] = df["Temperatura"].astype(str).str.replace(",", ".").astype(float)
    df["Umidade"] = df["Umidade"].astype(str).str.replace(",", ".").astype(float)
    #df = df.assign(Codigo=codigo)

    try:
        df["Codigo"] = codigo
        df["regiao"] = regiao
        df["uf"] = uf
        df["lat"] = latitude
        df["lon"] = longitude
    except Exception as e:
        print(df.columns)

        print(stringArquivo)

        print("Linhas:")
        for linha in linhas:
            print(linha)

        raise e

    myDF = df.copy()
    return myDF
    myDF = df.dropna()

    try: 
        returnVal = sqldf("""
        SELECT Mes, Ano, MEAN(lat) AS lat, MEAN(lon) as lon, Codigo, regiao, uf,
            MEAN(Umidade) AS "Umidade média mensal", MEAN(Temperatura) AS "Temperatura média mensal"
        FROM myDF
        GROUP BY Ano, Mes, Codigo, regiao, uf
        ORDER BY Ano, Mes
        """)
    
    except Exception as e:
        print(df.columns)

        print(stringArquivo)

        print("Linhas:")
        for linha in linhas:
            print(linha)

        raise e


    return returnVal


#geraDFSummary("./Anos/2021/INMET_CO_MT_A934_ALTO TAQUARI_01-01-2021_A_31-12-2021.CSV")

In [178]:
# achar arquivo especifico
for ano in os.listdir("./Anos/"):
    for arquivo in os.listdir("./Anos/%s/"%ano):

        if ano == "2020" and "C891" in arquivo:
            stringArquivo = "./Anos/%s/%s"%(ano, arquivo)
            print(stringArquivo)

./Anos/2020/INMET_SE_SP_C891_CRIOSFERA_01-01-2020_A_31-12-2020.CSV


In [181]:
# testar caso
geraDFSummary("./Anos/2020/INMET_SE_SP_C891_CRIOSFERA_01-01-2020_A_31-12-2020.CSV")


,Data,Umidade,Temperatura,Ano,Mes,Codigo,regiao,uf,lat,lon
0,2020/01/01,77.0,-13.3,2020,1,C891,SE,SP,-84.0,-79.494167
1,2020/01/01,77.0,-13.4,2020,1,C891,SE,SP,-84.0,-79.494167
2,2020/01/01,76.0,-13.8,2020,1,C891,SE,SP,-84.0,-79.494167
3,2020/01/01,NaN,NaN,2020,1,C891,SE,SP,-84.0,-79.494167
4,2020/01/01,69.0,-14.5,2020,1,C891,SE,SP,-84.0,-79.494167
...,...,...,...,...,...,...,...,...,...,...
8779,2020/12/31,63.0,-13.9,2020,12,C891,SE,SP,-84.0,-79.494167
8780,2020/12/31,68.0,-14.6,2020,12,C891,SE,SP,-84.0,-79.494167
8781,2020/12/31,68.0,-14.8,2020,12,C891,SE,SP,-84.0,-79.494167
8782,2020/12/31,65.0,-14.8,2020,12,C891,SE,SP,-84.0,-79.494167


In [176]:
j = 0
for ano in os.listdir("./Anos/"):
    for arquivo in os.listdir("./Anos/%s/"%ano):
        j+=1
print(j)

i = 0

listaDF = []

for ano in os.listdir("./Anos/"):
    for arquivo in os.listdir("./Anos/%s/"%ano):
        stringArquivo = "./Anos/%s/%s"%(ano, arquivo)
        #print(ano, arquivo, stringArquivo)
        df = geraDFSummary(stringArquivo)
        listaDF.append(df)
        i+=1

        print(i, "%.2f%%"%(100*i/j))
        
    
    

i

3495
1 0.03%
2 0.06%
3 0.09%
4 0.11%
5 0.14%
6 0.17%
7 0.20%
8 0.23%
9 0.26%
10 0.29%
11 0.31%
12 0.34%
13 0.37%
14 0.40%
15 0.43%
16 0.46%
17 0.49%
18 0.52%
19 0.54%
20 0.57%
21 0.60%
22 0.63%
23 0.66%
24 0.69%
25 0.72%
26 0.74%
27 0.77%
28 0.80%
29 0.83%
30 0.86%
31 0.89%
32 0.92%
33 0.94%
34 0.97%
35 1.00%
36 1.03%
37 1.06%
38 1.09%
39 1.12%
40 1.14%
41 1.17%
42 1.20%
43 1.23%
44 1.26%
45 1.29%
46 1.32%
47 1.34%
48 1.37%
49 1.40%
50 1.43%
51 1.46%
52 1.49%
53 1.52%
54 1.55%
55 1.57%
56 1.60%
57 1.63%
58 1.66%
59 1.69%
60 1.72%
61 1.75%
62 1.77%
63 1.80%
64 1.83%
65 1.86%
66 1.89%
67 1.92%
68 1.95%
69 1.97%
70 2.00%
71 2.03%
72 2.06%
73 2.09%
74 2.12%
75 2.15%
76 2.17%
77 2.20%
78 2.23%
79 2.26%
80 2.29%
81 2.32%
82 2.35%
83 2.37%
84 2.40%
85 2.43%
86 2.46%
87 2.49%
88 2.52%
89 2.55%
90 2.58%
91 2.60%
92 2.63%
93 2.66%
94 2.69%
95 2.72%
96 2.75%
97 2.78%
98 2.80%
99 2.83%
100 2.86%
101 2.89%
102 2.92%
103 2.95%
104 2.98%
105 3.00%
106 3.03%
107 3.06%
108 3.09%
109 3.12%
110 3.15%
111

3495

In [177]:
pd.concat(listaDF).to_csv("summary.csv", index=False)